In [ ]:
#cluster all participants 
import pandas as pd
import numpy as np
df=pd.read_csv('Feature.csv',index_col='iid')
data=df.drop(['expnum','race'],axis=1)
data=data.dropna()

def kmeans_cluster(data):
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=7, init='k-means++',n_init=30)
    return km.fit_predict(data)

data['cluster']=kmeans_cluster(data.drop(['gender'],axis=1))
data.to_csv('data.csv')

In [ ]:
def general_match():
#calculate the speed dating's match ratio between persons from same types and persons from different types
    df=pd.read_csv('Speed Dating Data.csv')
    man=df[(df.gender==1)][['iid','pid','match']]
    woman=df[(df.gender==0)][['iid','pid','match']]
    wdata=woman.dropna()
    mdata=man.dropna()
    wdf=pd.read_csv('data.csv')[df['gender']==0]
    mdf=pd.read_csv('data.csv')[df['gender']==1]
    m_same_sum=0
    m_same_match=0
    m_diff_sum=0
    m_diff_match=0
    w_same_sum=0
    w_same_match=0
    w_diff_sum=0
    w_diff_match=0
    for row in mdata.iterrows():
        if len(mdf[mdf['iid']==row[1][0]]) == 0 or len(wdf[wdf['iid']==row[1][1]])==0:
            continue
        r_n=mdf[mdf['iid']==row[1][0]].iloc[0].cluster
        c_n=wdf[wdf['iid']==row[1][1]].iloc[0].cluster
        if r_n==c_n:
            m_same_sum += 1
            if row[1][2]==1:
                m_same_match += 1
        else:
            m_diff_sum += 1
            if row[1][2]==1:
                m_diff_match += 1
                
    for row in wdata.iterrows():
        if len(wdf[wdf['iid']==row[1][0]]) == 0 or len(mdf[mdf['iid']==row[1][1]])==0:
            continue
        r_n=wdf[wdf['iid']==row[1][0]].iloc[0].cluster
        c_n=mdf[mdf['iid']==row[1][1]].iloc[0].cluster
        if r_n==c_n:
            w_same_sum += 1
            if row[1][2]==1:
                w_same_match += 1
        else:
            w_diff_sum += 1
            if row[1][2]==1:
                w_diff_match += 1
                
    return m_same_match/m_same_sum,m_diff_match/m_diff_sum,w_same_match/w_same_sum,w_diff_match/w_diff_sum
m_same_ratio,m_diff_ratio,w_same_ratio,w_diff_ratio=general_match()

In [ ]:
print(m_same_ratio,m_diff_ratio,w_same_ratio,w_diff_ratio)

In [ ]:
#cluster men participants and female participants separately 
import pandas as pd
import numpy as np
df=pd.read_csv('Feature.csv',index_col='iid')
man=df[df['gender']==1].drop(['expnum','gender','race'],axis=1)
woman=df[df['gender']==0].drop(['expnum','gender','race'],axis=1)
wdata=woman.dropna()
mdata=man.dropna()

def kmeans_cluster(data):
    from sklearn.cluster import KMeans
    from scipy.spatial.distance import cdist,pdist
    best=0
    result=list()
    for i in range(50):
        km = KMeans(n_clusters=7, init='k-means++',n_init=30)
        cent = km.fit(data).cluster_centers_
        k_euclid = cdist(data, cent,'euclidean')
        dist = np.min(k_euclid, axis=1)
        wcss = sum(dist**2)
        tss = sum(pdist(wdata)**2)/data.shape[0]
        bss = tss - wcss
        ratio = bss/tss
        if ratio>best:
            best=ratio
            result=km.predict(data)
    return result

wdata['cluster']=kmeans_cluster(wdata)
mdata['cluster']=kmeans_cluster(mdata)
wdata.to_csv('wdata.csv')
mdata.to_csv('mdata.csv')

In [ ]:
#calculate the features average values of participants from different types
#to avoid the bias results from extreme values, the average values calculated from 3 sigma areas.
w_feature=pd.DataFrame(data=np.zeros((7,len(list(wdata)))),columns=list(wdata),dtype=float)
for i in range(7):
    wgroup=wdata[wdata['cluster']==i]
    for j in range(len(list(wdata))):
        l=list(wgroup.iloc[:,j])
        l_inrange=list()
        for x in l: 
            if np.mean(l)-3*np.std(l)<= x <=np.mean(l)+3*np.std(l):
                l_inrange.append(x)
        w_feature.iloc[i,:].loc[list(wgroup)[j]]=np.mean(l_inrange)
m_feature=pd.DataFrame(data=np.zeros((7,len(list(mdata)))),columns=list(mdata),dtype=float)
for i in range(7):
    mgroup=mdata[mdata['cluster']==i]
    for j in range(len(list(wdata))):
        l=list(mgroup.iloc[:,j])
        l_inrange=list()
        for x in l: 
            if np.mean(l)-3*np.std(l)<= x <=np.mean(l)+3*np.std(l):
                l_inrange.append(x)
        m_feature.iloc[i,:].loc[list(mgroup)[j]]=np.mean(l_inrange)      
w_feature.to_csv('wfeature.csv')
m_feature.to_csv('mfeature.csv')

In [ ]:
#calculate the match numbers 
def get_match_sum():
    df=pd.read_csv('Speed Dating Data.csv')
    man=df[(df.gender==1) & (df.match==1)][['iid','pid']]
    woman=df[(df.gender==0) & (df.match==1)][['iid','pid']]
    wdata=woman.dropna()
    mdata=man.dropna()
    wdf=pd.read_csv('wdata.csv')
    mdf=pd.read_csv('mdata.csv')
    m_best_match=np.zeros((7,7))
    for row in mdata.iterrows():
        if len(mdf[mdf['iid']==row[1][0]]) == 0 or len(wdf[wdf['iid']==row[1][1]])==0:
            continue
        r_n=mdf[mdf['iid']==row[1][0]].iloc[0].cluster
        c_n=wdf[wdf['iid']==row[1][1]].iloc[0].cluster
        m_best_match[r_n][c_n] = m_best_match[r_n][c_n] + 1
    w_best_match=np.zeros((7,7))
    for row in wdata.iterrows():
        if len(wdf[wdf['iid']==row[1][0]]) == 0 or len(mdf[mdf['iid']==row[1][1]])==0:
            continue
        r_n=wdf[wdf['iid']==row[1][0]].iloc[0].cluster
        c_n=mdf[mdf['iid']==row[1][1]].iloc[0].cluster
        w_best_match[r_n][c_n] = w_best_match[r_n][c_n] + 1
    return m_best_match,w_best_match
m_best_match,w_best_match=get_match_sum()

In [ ]:
#calculate the dates numbers
def get_date_sum():
    df=pd.read_csv('Speed Dating Data.csv')
    man=df[(df.gender==1)][['iid','pid']]
    woman=df[(df.gender==0)][['iid','pid']]
    wdata=woman.dropna()
    mdata=man.dropna()
    wdf=pd.read_csv('wdata.csv')
    mdf=pd.read_csv('mdata.csv')
    m_date_sum=np.zeros((7,7))
    for row in mdata.iterrows():
        if len(mdf[mdf['iid']==row[1][0]]) == 0 or len(wdf[wdf['iid']==row[1][1]])==0:
            continue
        r_n=mdf[mdf['iid']==row[1][0]].iloc[0].cluster
        c_n=wdf[wdf['iid']==row[1][1]].iloc[0].cluster
        m_date_sum[r_n][c_n] += 1
    w_date_sum=np.zeros((7,7))
    for row in wdata.iterrows():
        if len(wdf[wdf['iid']==row[1][0]]) == 0 or len(mdf[mdf['iid']==row[1][1]])==0:
            continue
        r_n=wdf[wdf['iid']==row[1][0]].iloc[0].cluster
        c_n=mdf[mdf['iid']==row[1][1]].iloc[0].cluster
        w_date_sum[r_n][c_n] += 1
    return m_date_sum,w_date_sum
m_date_sum,w_date_sum=get_date_sum()

In [ ]:
#calculate the best match type
m_ratio=np.zeros((7,7))
for i in range(7):
    for j in range(7):
        m_ratio[i][j] = m_best_match[i][j]/m_date_sum[i][j]
w_ratio=np.zeros((7,7))
for i in range(7):
    for j in range(7):
        w_ratio[i][j] = w_best_match[i][j]/w_date_sum[i][j]
pd.DataFrame(m_ratio.argmax(axis=1)).to_csv('mmatch.csv')
pd.DataFrame(w_ratio.argmax(axis=1)).to_csv('wmatch.csv')

In [ ]:
#calculate the decision numbers
def get_dec_sum():
    df=pd.read_csv('Speed Dating Data.csv')
    man=df[(df.gender==1) & (df.dec_o==1)][['iid','pid']]
    woman=df[(df.gender==0) & (df.dec_o==1)][['iid','pid']]
    wdata=woman.dropna()
    mdata=man.dropna()
    wdf=pd.read_csv('wdata.csv')
    mdf=pd.read_csv('mdata.csv')
    m_best_match=np.zeros((7,7))
    for row in mdata.iterrows():
        if len(mdf[mdf['iid']==row[1][0]]) == 0 or len(wdf[wdf['iid']==row[1][1]])==0:
            continue
        r_n=mdf[mdf['iid']==row[1][0]].iloc[0].cluster
        c_n=wdf[wdf['iid']==row[1][1]].iloc[0].cluster
        m_best_match[r_n][c_n] = m_best_match[r_n][c_n] + 1
    w_best_match=np.zeros((7,7))
    for row in wdata.iterrows():
        if len(wdf[wdf['iid']==row[1][0]]) == 0 or len(mdf[mdf['iid']==row[1][1]])==0:
            continue
        r_n=wdf[wdf['iid']==row[1][0]].iloc[0].cluster
        c_n=mdf[mdf['iid']==row[1][1]].iloc[0].cluster
        w_best_match[r_n][c_n] = w_best_match[r_n][c_n] + 1
    return m_best_match,w_best_match
m_best_dec,w_best_dec=get_dec_sum()

In [ ]:
#calculate the most porpolar desicion type
m_ratio=np.zeros((7,7))
for i in range(7):
    for j in range(7):
        m_ratio[i][j] = m_best_dec[i][j]/m_date_sum[i][j]
w_ratio=np.zeros((7,7))
for i in range(7):
    for j in range(7):
        w_ratio[i][j] = w_best_dec[i][j]/w_date_sum[i][j]
pd.DataFrame(m_ratio.argmax(axis=1)).to_csv('mdec.csv')
pd.DataFrame(w_ratio.argmax(axis=1)).to_csv('wdec.csv')